In [4]:
!pip install pycryptodome

In [2]:
!pip install flask pyngrok pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 17.0 MB/s eta 0:00:00


In [ ]:
from flask import Flask, request, render_template_string
import hashlib
from pyngrok import conf, ngrok
import os

app = Flask(__name__)
os.environ["NGROK_AUTH_TOKEN"] = "2ujCdDkaOVBkBmrqIRJACyVCwRD_2nF1MMtFrfkdkjTpKxkJT"
conf.get_default().auth_token = os.environ["NGROK_AUTH_TOKEN"]

# Hàm băm SHA-256
def hash_sha256(data: bytes) -> str:
    sha256_hash = hashlib.sha256()
    sha256_hash.update(data)
    return sha256_hash.hexdigest()

# Hàm băm SHA-512
def hash_sha512(data: bytes) -> str:
    sha512_hash = hashlib.sha512()
    sha512_hash.update(data)
    return sha512_hash.hexdigest()

# HTML giao diện người dùng (giữ nguyên giao diện màu)
HTML = '''
<!DOCTYPE html>
<html lang="vi">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Băm dữ liệu bằng SHA-256 và SHA-512</title>
    <style>
        body {
            font-family: 'Segoe UI', sans-serif;
            background: linear-gradient(to right, #161717, #8b0404);
            min-height: 100vh;
            margin: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            padding: 20px;
            box-sizing: border-box;
        }

        .container {
            background: #141212cf;
            backdrop-filter: blur(5px);
            padding: 30px 40px;
            border-radius: 20px;
            box-shadow: 0 0 30px rgba(0, 0, 0, 0.6);
            width: 100%;
            max-width: 500px;
            color: #ffffff;
            box-sizing: border-box;
        }

        h2 {
            text-align: center;
            margin-bottom: 25px;
            color: #ffffff;
            font-size: 22px;
        }

        p {
            margin: 12px 0 6px;
            font-weight: 600;
        }

        input[type="file"],
        select {
            width: 100%;
            padding: 10px 12px;
            margin-top: 6px;
            border-radius: 10px;
            border: 1px solid #332626;
            background-color: #1e1e1e;
            color: #ffffff;
            font-size: 14px;
        }

        button {
            width: 100%;
            padding: 12px;
            border: none;
            border-radius: 10px;
            background: linear-gradient(to right, #000000, #b30606);
            color: white;
            font-weight: bold;
            font-size: 16px;
            cursor: pointer;
            margin-top: 20px;
            transition: background 0.3s ease;
        }

        button:hover {
            background: linear-gradient(to right, #000000, #1037aa);
        }

        .result-box {
            background-color: #ffffff;
            border: 1px solid #ccc;
            border-radius: 8px;
            padding: 15px;
            margin-top: 25px;
            word-break: break-word;
            font-family: 'Courier New', monospace;
            color: #333;
            font-size: 0.9em;
        }

        .result-box strong {
            display: block;
            margin-bottom: 5px;
            color: #555;
        }

        .error {
            color: red;
            font-weight: bold;
            margin-top: 15px;
            text-align: center;
        }

        #file-name {
            font-size: 13px;
            color: #ccc;
            margin-top: 5px;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Băm dữ liệu bằng SHA-256 / SHA-512</h2>
        <form method="POST" enctype="multipart/form-data">
            <p>Chọn file:
            <label for="file" id="file-label" style="background:#940000; padding:10px; border-radius:8px; display:block; text-align:center; cursor:pointer;">Chọn File</label>
            <input type="file" id="file" name="file" style="display:none;" required>
            <div id="file-name"></div>
            </p>

            <p>Thuật toán băm:
            <select name="algorithm" required>
                <option value="sha256">SHA-256</option>
                <option value="sha512">SHA-512</option>
            </select></p>

            <button type="submit">Thực hiện băm</button>
        </form>

        {% if original_hashed_result %}
            <div class="result-box">
                <strong>Kết quả băm gốc ({{ algorithm }}):</strong>
                <p>{{ original_hashed_result }}</p>
            </div>
            <div class="result-box">
                <strong>Kết quả băm đã sửa đổi ({{ algorithm }} - thêm một ký tự):</strong>
                <p>{{ modified_hashed_result }}</p>
            </div>
        {% endif %}

        {% if error %}
            <div class="error">{{ error }}</div>
        {% endif %}
    </div>

    <script>
        const fileInput = document.getElementById('file');
        const fileNameDiv = document.getElementById('file-name');

        fileInput.addEventListener('change', function() {
            if (fileInput.files.length > 0) {
                fileNameDiv.textContent = 'Đã chọn: ' + fileInput.files[0].name;
            } else {
                fileNameDiv.textContent = '';
            }
        });
    </script>
</body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def home():
    error = None
    original_hashed_result = None
    modified_hashed_result = None
    algorithm_used = None

    if request.method == 'POST':
        if 'file' not in request.files:
            error = "Không có file nào được chọn."
        else:
            f = request.files['file']
            if f.filename == '':
                error = "Không có file nào được chọn."
            else:
                algorithm = request.form.get('algorithm', 'sha256')
                original_data = f.read()
                modified_data = original_data + b'\n'

                try:
                    if algorithm == 'sha256':
                        original_hashed_result = hash_sha256(original_data)
                        modified_hashed_result = hash_sha256(modified_data)
                        algorithm_used = "SHA-256"
                    elif algorithm == 'sha512':
                        original_hashed_result = hash_sha512(original_data)
                        modified_hashed_result = hash_sha512(modified_data)
                        algorithm_used = "SHA-512"
                    else:
                        error = "Thuật toán băm không hợp lệ."
                except Exception as e:
                    error = f"Lỗi trong quá trình băm: {e}"

    return render_template_string(
        HTML,
        original_hashed_result=original_hashed_result,
        modified_hashed_result=modified_hashed_result,
        algorithm=algorithm_used,
        error=error
    )

try:
    ngrok.kill()
    public_url = ngrok.connect(5000).public_url
    print(f"📌 Truy cập web tại: {public_url}")
except Exception as e:
    print(f"Lỗi khi khởi tạo ngrok: {e}")
    public_url = "Ngrok không thể khởi tạo."

app.run(port=5000, use_reloader=False)


📌 Truy cập web tại: https://93b7-34-138-215-91.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/May/2025 12:02:03] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 12:02:03] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 12:06:37] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 12:09:07] "GET / HTTP/1.1" 200 -


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
